In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

RANDOM_STATE = 42

from google.colab import files
uploaded = files.upload()

df = pd.read_csv("train.csv")
df.head()

Saving train.csv to train (3).csv


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Была выбрана метрика: F1-score, так как она учитывает как полноту, так и точность, что важно в несбалансированных задачах классификации, когда ошибки одного типа могут быть более критичны (например, не спасти выжившего). Более того, ROC-AUC позволяет понять качество модели по всем порогам вероятности, что полезно для анализа бинарной задачи.

In [3]:
df = df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])

df["Sex"] = LabelEncoder().fit_transform(df["Sex"])  # male = 1, female = 0
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])
df["Embarked"] = LabelEncoder().fit_transform(df["Embarked"])

imputer = SimpleImputer(strategy="median")
df[["Age", "Fare"]] = imputer.fit_transform(df[["Age", "Fare"]])

X = df.drop("Survived", axis=1)
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [4]:
dummy = DummyClassifier(strategy="most_frequent", random_state=RANDOM_STATE)
dummy.fit(X_train, y_train)
y_dummy_pred = dummy.predict(X_test)

print("Dummy Accuracy:", accuracy_score(y_test, y_dummy_pred))
print("Dummy F1:", f1_score(y_test, y_dummy_pred))
print("Dummy ROC-AUC:", roc_auc_score(y_test, y_dummy_pred))



Dummy Accuracy: 0.5865921787709497
Dummy F1: 0.0
Dummy ROC-AUC: 0.5


Теперь оценим качество каждой ML-модели с помощью выбранной метрики и определим наиболее подходящую.

In [7]:
results = {}

lr = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
y_proba_lr = lr.predict_proba(X_test)[:, 1]

results['LogisticRegression'] = {
    'Accuracy': accuracy_score(y_test, y_pred_lr),
    'Precision': precision_score(y_test, y_pred_lr),
    'Recall': recall_score(y_test, y_pred_lr),
    'F1': f1_score(y_test, y_pred_lr),
    'ROC-AUC': roc_auc_score(y_test, y_proba_lr)
}

rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

results['RandomForest'] = {
    'Accuracy': accuracy_score(y_test, y_pred_rf),
    'Precision': precision_score(y_test, y_pred_rf),
    'Recall': recall_score(y_test, y_pred_rf),
    'F1': f1_score(y_test, y_pred_rf),
    'ROC-AUC': roc_auc_score(y_test, y_proba_rf)
}

xgb = XGBClassifier(eval_metric='logloss', random_state=RANDOM_STATE)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)
y_proba_xgb = xgb.predict_proba(X_test)[:, 1]

results['XGBoost'] = {
    'Accuracy': accuracy_score(y_test, y_pred_xgb),
    'Precision': precision_score(y_test, y_pred_xgb),
    'Recall': recall_score(y_test, y_pred_xgb),
    'F1': f1_score(y_test, y_pred_xgb),
    'ROC-AUC': roc_auc_score(y_test, y_proba_xgb)
}

cat = CatBoostClassifier(verbose=0, random_state=RANDOM_STATE)
cat.fit(X_train, y_train)
y_pred_cat = cat.predict(X_test)
y_proba_cat = cat.predict_proba(X_test)[:, 1]

results['CatBoost'] = {
    'Accuracy': accuracy_score(y_test, y_pred_cat),
    'Precision': precision_score(y_test, y_pred_cat),
    'Recall': recall_score(y_test, y_pred_cat),
    'F1': f1_score(y_test, y_pred_cat),
    'ROC-AUC': roc_auc_score(y_test, y_proba_cat)
}

In [8]:
results_df = pd.DataFrame(results).T
print(results_df.sort_values(by='F1', ascending=False))

                    Accuracy  Precision    Recall        F1   ROC-AUC
RandomForest        0.821229   0.808824  0.743243  0.774648  0.898842
CatBoost            0.826816   0.852459  0.702703  0.770370  0.893694
LogisticRegression  0.810056   0.785714  0.743243  0.763889  0.882368
XGBoost             0.798883   0.756757  0.756757  0.756757  0.862420


Исходя из результатов, я остановлюсь на модели: "RandomForest", ведь она имеет наивысший ROC-AUC (0.8988), что говорит о её лучшей способности различать положительные и отрицательные классы.

Также она демонстрирует наивысший F1-скор (0.7746), что означает лучшую сбалансированность между точностью и полнотой, что особенно важно в задачах классификации, где нужно минимизировать как ложные положительные, так и ложные отрицательные срабатывания.

Таким образом, RandomForest показывает наилучшие общие результаты по большинству важных метрик, что делает её наиболее подходящей для данного набора данных.